# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [16]:
# 1) Seu código aqui
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
r = requests.get(url)
r.status_code
data_json = r.json()
estados_df = pd.DataFrame(data_json['items'])

In [55]:
# 2) Seu código aqui
estados_df['status'].value_counts()

Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [59]:
# 3) Seu código aqui
ano_status = estados_df['data_status'].str.split('/', expand=True)[2]
ano_status

0     2002
1     2012
2     2016
3     2011
4     2012
5     2009
6     2012
7     2012
8     2012
9     2002
10    2013
11    2013
12    2013
13    2011
14    2012
15    2013
16    2010
17    2012
18    2014
19    2014
20    2012
21    2008
22    2009
23    2002
24    2013
25    2012
26    2011
27    2006
28    2009
29    2011
30    2011
31    2006
32    2009
33    2016
34    2008
35    2014
36    2013
37    2009
38    2016
39    2014
40    2010
41    2002
42    2009
43    2012
44    2009
45    2016
46    2012
47    2010
48    2011
49    2009
50    2013
51    2009
Name: 2, dtype: object

In [60]:
# 4
ano_status.value_counts()

2012    11
2009     9
2013     7
2011     6
2002     4
2016     4
2014     4
2010     3
2008     2
2006     2
Name: 2, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [65]:
#1) Seu código aqui
def handle_api(uf: str, tipo:str) -> pd.DataFrame():
    url = f"http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf.upper()}&tipo_interessado={tipo.capitalize()}"
    r = requests.get(url)
    r.status_code
    data_json = r.json()
    return pd.DataFrame(data_json['items'])

In [90]:
# 2) Seu código aqui
pd_minas = handle_api("mg", "estado")
minas_count = pd_minas[pd_minas['status'] == "Arquivado por decurso de prazo"].count()
minas_count[minas_count['status']]

1

In [97]:
# 3) Seu código aqui
pd_bahia = handle_api("ba", "município")
pd_bahia['interessado'].value_counts()

Camaçari                  19
Salvador                  17
Luís Eduardo Magalhães    15
Vitória da Conquista      14
Amargosa                  14
                          ..
Monte Santo                1
Cravolândia                1
Taperoá                    1
Nova Ibiá                  1
Barro Alto                 1
Name: interessado, Length: 289, dtype: int64

In [98]:
# 4) Seu código aqui
pd_bahia = handle_api("ba", "estado")

In [101]:
pd_bahia.to_csv("dados_bahia.csv")